# Use Vertex AI Vector Search to Recommend Similar Products

## Overview

This notebook demonstrates how to use the Vertex AI Vector Search Service. It is a high-scale, low-latency solution, to find similar vectors (or more specifically "embeddings") for a large dataset. Moreover, it is a fully managed offering, further reducing operational overhead. It is built upon [Approximate Nearest Neighbor (ANN) technology](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) developed by Google Research.

## Terminology

This section reviews some key terminology related to vector search.

- **Vector**: A vector is a mathematical object that has both a magnitude and a direction. It is often represented as an array of numbers. In the context of machine learning, a vector is often used to represent a feature or an embedding.
- **Embedding**: An embedding is a mathematical representation of an object, such as a word, sentence, image, or sound. In the context of machine learning, an embedding is often used to represent a feature. For this lab, product embeddings are used to represent products. Each product embedding is a 768-dimension vector meaning 768 numbers are used to represent the product.
- **Vector Search**: Vector search is the process of finding similar vectors (or embeddings) for a given query vector. It is often used to find or recommend similar products, images, or documents.
- **Index**: An index is a data structure that is used to store and retrieve embeddings. It is optimized for fast search and retrieval of similar embeddings.

### Objective

In this notebook, you learn how to create Approximate Nearest Neighbor (ANN) Index in Vertex AI (formerly known as AI Platform), query against indexes, and validate the performance of the index. 

The steps performed include:

* Create an Index
* Create an IndexEndpoint
* Deploy an Index to an IndexEndpoint
* Perform a vector search query
* Compare the results with the ground truth

The following diagram illustrates the different services involved:

![](assets/overview.png)

- The product embeddings are stored in Google Cloud Storage. The embeddings have been generated using [Vertex AI embeddings for text](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text-embeddings) using the product names
- Each Vector Search Index is created in Vertex AI using the embeddings.
- There are two indexes created, one using the Approximate Nearest Neighbor algorithm (ANN) and the other using the Brute Force algorithm.
    - Nearest neighbor refers to finding the closest embedding to a given query embedding.
    - The ANN algorithm is optimized for fast search and retrieval of similar embeddings although it is not guaranteed to find the best solution, especially for large datasets. 
    - The Brute Force algorithm will find the true nearest neighbor but is less efficient and not recommended for production. By comparing the results of the ANN and Brute Force algorithms, you can validate the performance of the ANN algorithm.
- To query the indexes, an IndexEndpoint is created and the indexes are deployed to the endpoint.
- IndexEndpoints can be deployed in private or public mode. In this lab, the IndexEndpoints are deployed in private mode which requires VPC peering to query the endpoints. The lab environment has been set up with VPC peering so the notebook instance can query the endpoints.

*Note*: The index creation and deployment can take up to 30 minutes to complete. Because of this, they have automatically been deployed when you started this lab to save you time waiting. You will understand all of the code involved in creating and deploying the index before performing real-time queries against the index endpoints.

## Understanding the sample data

In this lab, you will use [TheLook dataset](https://console.cloud.google.com/marketplace/product/bigquery-public-data/thelook-ecommerce) which has [products](bigquery-public-data.thelook_ecommerce.products) table with about 30,000 rows of synthetic product data for a fictitious e-commerce clothing site.

![](assets/Thelook.png)

From this table, a `product-embeddings.json` file has been prepared for this lab which includes 5000 product embeddings.

This file is in JSONL (JSON lines) format and each row has an `id` for the product id, `name` for the product name, and `embedding` for the embedding vector of the product name in 768 dimensions which was generated using [Vertex AI Embeddings for Text](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings). A sample row from the file is as follows (where the `embedding` vector shows only 3 of the 768 dimensions):

```json
{"id":"19536","name":"original penguin men's pro-bro mock sweater","embedding":[0.015607465989887714,0.0183266568928957,0.080682516098022461,...]}
```

The text embeddings represent the meaning of the clothing product names. In this lab, you will use Vertex AI Vector Search to complete a [semantic](https://en.wikipedia.org/wiki/Semantic_search) search](https://en.wikipedia.org/wiki/Semantic_search) of the items. This sample code can be used as a basis for other simple recommendation systems where you can quickly find other items similar to a given one.

## Installation

Install the following Python packages required to execute this notebook:

- `google-cloud-aiplatform` - The official Python client library for Vertex AI.
- `google-cloud-storage` - The official Python client library for Google Cloud Storage.
- `grpcio-tools` - The gRPC tools for Python. gRPC is a high-performance, open-source universal remote procedure call (RPC) framework.

You can ignore any pip errors about other dependencies as they do not impact this notebook.

In [ ]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform==1.42.1 \
                         google-cloud-storage \
                         grpcio-tools

## Before you begin
#### Get your project ID and Number

In [ ]:
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]

# Retrieve the project number
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

### Prepare a VPC network
To reduce any network overhead that might lead to unnecessary increase in overhead latency, it is best to call the ANN endpoints from your VPC via a direct [VPC Peering](https://cloud.google.com/vertex-ai/docs/general/vpc-peering) connection. 
  * The following section describes how to setup a VPC Peering connection if you don't have one. 
  * This is a one-time initial setup task. You can also reuse existing VPC network and skip this section.

In [ ]:
VPC_NETWORK = "ai-net"

## Make sure the following cells are run from inside the VPC network that you created in the previous step.

* **WARNING:** The MatchingIndexEndpoint.match method (to create online queries against your deployed index) has to be executed in a Vertex AI Workbench notebook instance that is created with the following requirements:
  * **In the same region as where your ANN service is deployed** (for example, if you set `REGION = "us-central1"` as same as the tutorial, the notebook instance has to be in `us-central1`).
  * **Make sure you select the VPC network you created for ANN service** (instead of using the "default" one). That is, you will have to create the VPC network below and then create a new notebook instance that uses that VPC.  
  * If you run it in the colab or a Vertex AI Workbench notebook instance in a different VPC network or region, "Create Online Queries" section will fail.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = "{BUCKET_PLACEHOLDER}"

## Prepare the data

The GloVe dataset consists of a set of pre-trained embeddings. The embeddings are split into a "train" split, and a "test" split.
We will create a vector search index from the "train" split, and use the embedding vectors in the "test" split as query vectors to test the vector search index.

**Note:** While the data split uses the term "train", these are pre-trained embeddings and therefore are ready to be indexed for search. The terms "train" and "test" split are used just to be consistent with machine learning terminology.

Download the GloVe dataset.


In [ ]:
! gsutil cp "gs://ca-lab-assets/vector-search/product-embeddings.json" .

## Create Indexes


### Create ANN Index (for Production Usage)

In [ ]:
DIMENSIONS = 100
DISPLAY_NAME = "glove_100_1"
DISPLAY_NAME_BRUTE_FORCE = DISPLAY_NAME + "_brute_force"

Create the ANN index configuration:

To learn more about configuring the index, see [Input data format and structure](https://cloud.google.com/vertex-ai/docs/matching-engine/match-eng-setup/format-structure).


In [ ]:
import os

from google.cloud import aiplatform

aiplatform.init(location="us-central1", staging_bucket=BUCKET_URI)

In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=50,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=50,
    leaf_nodes_to_search_percent=7,
    description="Glove 100 ANN index",
    labels={"label_name": "label_value"},
)

In [ ]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

Using the resource name, you can retrieve an existing MatchingEngineIndex.

In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

### Create Brute Force Index (for Ground Truth)

The brute force index uses a naive brute force method to find the nearest neighbors. This method is not fast or efficient. Hence brute force indices are not recommended for production usage. They are to be used to find the "ground truth" set of neighbors, so that the "ground truth" set can be used to measure recall of the indices being tuned for production usage. To ensure an apples to apples comparison, the `distanceMeasureType` and `dimensions` of the brute force index should match those of the production indices being tuned.

Create the brute force index configuration:

In [ ]:
brute_force_index = aiplatform.MatchingEngineIndex.create_brute_force_index(
    display_name=DISPLAY_NAME_BRUTE_FORCE,
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    dimensions=DIMENSIONS,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    description="Glove 100 index (brute force)",
    labels={"label_name": "label_value"},
)

In [ ]:
INDEX_BRUTE_FORCE_RESOURCE_NAME = brute_force_index.resource_name
INDEX_BRUTE_FORCE_RESOURCE_NAME

In [ ]:
brute_force_index = aiplatform.MatchingEngineIndex(
    index_name=INDEX_BRUTE_FORCE_RESOURCE_NAME
)

## Update Indexes

Create incremental data file.


In [ ]:
with open("glove100_incremental.json", "w") as f:
    index = 0
    f.write(
        json.dumps(
            {
                "id": str(index),
                "embedding": [str(0) for _ in train[index]],
                "restricts": [
                    {
                        "namespace": "class",
                        "allow_list": ["even" if index % 2 == 0 else "odd"],
                    }
                ],
            }
        )
        + "\n"
    )

Copy the incremental data file to a new subdirectory.


In [ ]:
EMBEDDINGS_UPDATE_URI = f"{BUCKET_URI}/matching-engine/incremental/"

In [ ]:
! gsutil cp glove100_incremental.json {EMBEDDINGS_UPDATE_URI}

Create update index request


In [ ]:
tree_ah_index = tree_ah_index.update_embeddings(
    contents_delta_uri=EMBEDDINGS_UPDATE_URI,
)

In [ ]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

## Create an IndexEndpoint with VPC Network

In [ ]:
VPC_NETWORK = "ai-net"
VPC_NETWORK_FULL = f"projects/{PROJECT_NUMBER}/global/networks/{VPC_NETWORK}"
VPC_NETWORK_FULL

In [ ]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="product_index_endpoint",
    description="product index endpoint",
    network=VPC_NETWORK_FULL,
)

In [ ]:
INDEX_ENDPOINT_NAME = my_index_endpoint.resource_name
INDEX_ENDPOINT_NAME

## Deploy Indexes

### Deploy ANN Index

In [ ]:
DEPLOYED_INDEX_ID = "product_deployed_index"

In [ ]:
my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

### Deploy Brute Force Index

In [ ]:
DEPLOYED_BRUTE_FORCE_INDEX_ID = "product_brute_force_deployed_index"

In [ ]:
my_index_endpoint = my_index_endpoint.deploy_index(
    index=brute_force_index, deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID
)

my_index_endpoint.deployed_indexes

## Confirm Index Deployment

If the indexes are still deploying, you can learn more about [Vertex AI Vector Search](https://cloud.google.com/vertex-ai/docs/matching-engine/overview) while waiting.

## Create Online Queries

After you built your indexes, you may query against the deployed index through the online querying gRPC API (Match service) within the virtual machine instances from the same region (for example 'us-central1' in this tutorial).

The `filter` parameter is an optional way to filter for a subset of embeddings. In this case, only embeddings that have the `class` set as `even` are returned.

In [ ]:
import json

# build dictionaries for product names and embeddings
product_names = {}
product_embeddings = {}
with open("product-embeddings.json") as f:
    for l in f.readlines():
        p = json.loads(l)
        id = p["id"]
        product_names[id] = p["name"]
        product_embeddings[id] = p["embedding"]

In [ ]:
# get the embedding for ID 17268 "hurley juniors supersuede beachrider boardshort"
# you can also try with other IDs such as 18090, 19536 and 11863
query_embedding = product_embeddings["12711"]

In [ ]:
index_endpoints = aiplatform.MatchingEngineIndexEndpoint.list()
index_endpoint = None
for i in index_endpoints:
    if "brute-force" not in i.display_name:
        index_endpoint = i
        break
        
index_endpoint

In [ ]:
# Test query

DEPLOYED_INDEX_ID = 'product_deployed_index'
NUM_NEIGHBOURS = 10 # The number of nearest neighbors to be retrieved

# Execute the request
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=[query_embedding],
    num_neighbors=NUM_NEIGHBOURS,
)

response

In [ ]:
# show the results
for idx, neighbor in enumerate(response[0]):
    print(f"{neighbor.distance:.2f} {product_names[neighbor.id]}")

### Compute Recall

Use the deployed brute force Index as the ground truth to calculate the recall of ANN Index. Note that you can run multiple queries in a single match call.

In [ ]:
# Retrieve nearest neighbors for both the tree-AH index and the brute-force index
tree_ah_response_test = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=list(test),
    num_neighbors=NUM_NEIGHBOURS,
)
brute_force_response_test = my_index_endpoint.match(
    deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID,
    queries=list(test),
    num_neighbors=NUM_NEIGHBOURS,
)

In [ ]:
# Calculate recall by determining how many neighbors were correctly retrieved as compared to the brute-force option.
recalled_neighbors = 0
for tree_ah_neighbors, brute_force_neighbors in zip(
    tree_ah_response_test, brute_force_response_test
):
    tree_ah_neighbor_ids = [neighbor.id for neighbor in tree_ah_neighbors]
    brute_force_neighbor_ids = [neighbor.id for neighbor in brute_force_neighbors]

    recalled_neighbors += len(
        set(tree_ah_neighbor_ids).intersection(brute_force_neighbor_ids)
    )

recall = recalled_neighbors / len(
    [neighbor for neighbors in brute_force_response_test for neighbor in neighbors]
)

print("Recall: {}".format(recall))